In [1]:
import bs4
import requests
import pandas as pd
import unicodedata
import re

In [2]:
urls = []
names = []
addresses = []
ids = []
photos = []
average_prices = []
q1_prices = []
q2_prices = []
q3_prices = []
q4_prices = []

for page_number in range(1, 133):
    URL = f"https://www.yazlikcim.com.tr/tum-kiralik-villalar/{page_number}"
    response = requests.get(URL)
    response.encoding = "utf-8"  
    html_parsed = bs4.BeautifulSoup(response.text, "html.parser")
    
    villa_names  = [villa_name.getText().strip() for villa_name in html_parsed.select("div.vadi h4")]
    villa_addresses = [villa_address.getText().strip() for villa_address in html_parsed.select("div.vadi .vyeri")]
    villa_urls = [villa_url["href"] for villa_url in html_parsed.select("div.resimtarafi a") if "upload" not in villa_url["href"]]
    villa_ids = [villa_name.lower().replace(" ", "").strip() for villa_name in villa_names]
    for all_photo_urls in html_parsed.select(".loopcont .resimtarafi .loopgallerytekcont"):
        villa_photos = tuple(photo_urls["href"] for photo_urls in all_photo_urls.select("a"))
        photos.append(villa_photos)

    all_prices = [list(map(lambda x: int(x.strip()), villa_prices.getText().replace("ARALIĞINDA", "").replace("₺", "").replace(",", "").split("-"))) for villa_prices in html_parsed.select(".fiyatvstarafi")]
    avg_prices = list(map(lambda x: sum(x) // len(x), all_prices))
    page_q1_prices = list(map(lambda x: x*0.14, avg_prices))
    page_q2_prices = list(map(lambda x: x*0.32, avg_prices))
    page_q3_prices = list(map(lambda x: x*0.37, avg_prices))
    page_q4_prices = list(map(lambda x: x*0.17, avg_prices))
    
    urls.extend(villa_urls)
    names.extend(villa_names)
    addresses.extend(villa_addresses)
    ids.extend(villa_ids)
    average_prices.extend(avg_prices)
    q1_prices.extend(page_q1_prices)
    q2_prices.extend(page_q2_prices)
    q3_prices.extend(page_q3_prices)
    q4_prices.extend(page_q4_prices)
    
    
    if page_number % 10 == 0:
        print(f"{page_number} pages have been check!")

print(f"\n---OPERATION HAS BEEN COMPLETED!---")
print(f"{page_number} pages have been checked!")

10 pages have been check!
20 pages have been check!
30 pages have been check!
40 pages have been check!
50 pages have been check!
60 pages have been check!
70 pages have been check!
80 pages have been check!
90 pages have been check!
100 pages have been check!
110 pages have been check!
120 pages have been check!
130 pages have been check!

---OPERATION HAS BEEN COMPLETED!---
132 pages have been checked!


In [3]:
guests = []
bedrooms = []
bathrooms = []
latitudes = []
longitudes = []
descriptions = []

URL_ERRORS = []
villa_number = 0
for URL in urls:
    try:
        response = requests.get(URL)
        response.encoding = "utf-8" 
        html_parsed = bs4.BeautifulSoup(response.text, "html.parser")
        villa_number += 1
    except:
        print(f"{URL} : URL ERROR!")
        URL_ERRORS.append(URL)
        
    try:
        guest = int(html_parsed.select("#gbtablo td")[1].getText().strip()[0])
    except:
        print(f"{URL} : GUEST NUMBER ERROR!")
        guest = None
        
    try:
        bedroom = int(html_parsed.select("#gbtablo td")[3].getText().strip()[0])
    except:
        print(f"{URL} : BEDROOM NUMBER ERROR!")
        bedroom = None
    
    try:
        bathroom = int(html_parsed.select("#gbtablo td")[5].getText().strip()[0])
    except:
        print(f"{URL} : BATHROOM NUMBER ERROR!")
        bathroom = None
        
    try:
        description = "\n".join([unicodedata.normalize("NFKD" ,txt.getText()) for txt in html_parsed.select_one("#contyazi").select("p")]).strip()
    except:
        print(f"{URL} : DESCRIPTION ERROR!")
        description = None
        
    try:
        pattern = r'center:(.*?)]'
        latitude, longitude = tuple(map(float, re.search(pattern, str(html_parsed)).group(0).strip()[9:-1].split(",")))
    except:
        print(f"{URL} : LOCATION ERROR!")
        latitude = None
        longitude = None
        
    guests.append(guest)
    bedrooms.append(bedroom)
    bathrooms.append(bathroom)
    descriptions.append(description)
    latitudes.append(latitude)
    longitudes.append(longitude)
     
    
    if villa_number % 75 == 0:
        print(f"{villa_number} villas have been completed!")

print(f"THE OPERATION IS COMPLETED!")
print(f"TOTAL VILLA NUMBER: {villa_number}")

75 villas have been completed!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-jasmin-sarbelen : LOCATION ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-fauna : LOCATION ERROR!
150 villas have been completed!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-sude-kordere : LOCATION ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-eslem-kordere : LOCATION ERROR!
225 villas have been completed!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-harun-koredere : LOCATION ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-alya-kordere : LOCATION ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-white-rock-islamlar : LOCATION ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-golden-sirena : LOCATION ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-sla : GUEST NUMBER ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-sla : BEDROOM NUMBER ERROR!
https://www.yazlikcim.com.tr/kiralik-villalar/villa-sla : BATHROOM NUMBER ER

In [5]:
data_raw = {
    "ID": ids,
    "Name": names,
    "Address": addresses,
    "Guest Number": guests,
    "Bedroom Number": bedrooms,
    "Bathroom Number":bathrooms,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "Description": descriptions,
    "Photo URLs": photos,
    "Average Price": average_prices,
    "Q1 Price": q1_prices,
    "Q2 Price": q2_prices,
    "Q3 Price": q3_prices,
    "Q4 Price": q4_prices
}

df_raw = pd.DataFrame(data_raw)
df_raw.set_index("ID", inplace=True)
df_raw.to_excel("yazlikcim_raw.xlsx")

data_info = {
    "ID": ids,
    "Name": names,
    "Address": addresses,
    "Guest Number": guests,
    "Bedroom Number": bedrooms,
    "Bathroom Number":bathrooms,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "Description": descriptions,
    "Photo URLs": photos  
}

df_info = pd.DataFrame(data_info)
df_info.set_index("ID", inplace=True)
df_info.to_excel("yazlikcim_info.xlsx")

data_financial = {
    "ID": ids,
    "Average Price": average_prices,
    "Q1 Price": q1_prices,
    "Q2 Price": q2_prices,
    "Q3 Price": q3_prices,
    "Q4 Price": q4_prices   
}

df_financial = pd.DataFrame(data_financial)
df_financial.set_index("ID", inplace=True)
df_financial.to_excel("yazlikcim_financial.xlsx")